In [1]:
import sys
import os
import time

import matplotlib
# Force matplotlib to not use any Xwindows backend.
matplotlib.use('Agg') # https://stackoverflow.com/a/3054314
import matplotlib.pyplot as plt
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from registration_utilities import *
from annotation_utilities import *
from metadata import *
from data_manager import *

from aligner_v3 import Aligner

Setting environment for Precision WorkStation
{'DEMO998': 0.46}


ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
No vtk
No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
Seems you are using operation INIs to provide cropbox.
No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
Seems you are using operation INIs to provide cropbox.


In [3]:
brain_f_spec = load_json('../../demo/demo_fixed_brain_spec_12N.json')
brain_m_spec = load_json('../../demo/demo_moving_brain_spec_12N.json')
registration_setting = 7
use_simple_global = True


In [4]:
structures_f = brain_f_spec['structure']
if isinstance(structures_f, str):
    structures_f = [structures_f]
    
structures_m = brain_m_spec['structure']
if isinstance(structures_m, str):
    structures_m = [structures_m]

if brain_f_spec['vol_type'] == 'annotationAsScore': # If Neurolucida annotation
    fixed_surroundings_have_positive_value = True
    fixed_use_surround = True
elif brain_f_spec['vol_type'] == 'score': # If detection score map
    fixed_surroundings_have_positive_value = False
    fixed_use_surround = False
else:
    raise


In [5]:
alignment_spec = dict(stack_m=brain_m_spec, stack_f=brain_f_spec, warp_setting=registration_setting)

brain_m_spec0 = brain_m_spec.copy()
brain_m_spec0.pop("structure")
brain_f_spec0 = brain_f_spec.copy()
brain_f_spec0.pop("structure")
simpleGlobal_alignment_spec = dict(stack_m=brain_m_spec0, stack_f=brain_f_spec0, warp_setting=0)

aligner_parameters = generate_aligner_parameters_v2(alignment_spec=alignment_spec, 
                                                    structures_m=structures_m,
                                                   fixed_structures_are_sided=True,
 fixed_surroundings_have_positive_value=fixed_surroundings_have_positive_value,
                                                   fixed_use_surround=fixed_use_surround)

aligner = Aligner(aligner_parameters['volume_fixed'], 
                  aligner_parameters['volume_moving'], 
                  labelIndexMap_m2f=aligner_parameters['label_mapping_m2f'])

aligner.compute_gradient(smooth_first=True)
aligner.set_label_weights(label_weights=aligner_parameters['label_weights_m'])


upstream_warp_id                                                               None
transform_type                                                                rigid
grad_computation_sample_number                                               100000
grid_search_sample_number                                                      1000
std_tx_um                                                                        50
std_ty_um                                                                        50
std_tz_um                                                                        50
std_theta_xy_degree                                                              10
surround_weight                                                             inverse
regularization_weight                                                           NaN
terminate_thresh_trans                                                         0.01
terminate_thresh_rot                                                        

Prior structure/index map not found. Generating a new one.
Loaded moving volumes: [u'12N', u'12N_surround_200um'].
Prior structure/index map not found. Generating a new one.
Loaded fixed volumes: [u'12N'].
Computing structure sizes: 0.00 s
Label weights not set, default to 1 for all structures.
Regularization weights not set, default to 0.
indices_f: set([1])


In [8]:
if use_simple_global:
    T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol = np.loadtxt(os.path.join(DATA_ROOTDIR, 'CSHL_simple_global_registration', brain_f_spec['name'] + '_T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol.txt'))
    aligner.set_initial_transform(T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol)
    aligner.set_centroid(centroid_m='structure_centroid', centroid_f='centroid_m')
else:
    T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])
    aligner.set_initial_transform(T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol)
    aligner.set_centroid(centroid_m='structure_centroid', centroid_f='structure_centroid')

Set initial transform to [1.00000000e+00 0.00000000e+00 0.00000000e+00 1.03245321e+03
 0.00000000e+00 1.00000000e+00 0.00000000e+00 4.14338454e+02
 0.00000000e+00 0.00000000e+00 1.00000000e+00 4.48000000e+02].
centroid_m: [1230.83790861  481.74054169  446.5       ], centroid_f: [1230.83790861  481.74054169  446.5       ]


In [9]:
grid_search_T = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])
init_T = compose_alignment_parameters([T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol, grid_search_T])
print init_T

[[1.00000000e+00 0.00000000e+00 0.00000000e+00 1.03245321e+03]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 4.14338454e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 4.48000000e+02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [10]:
for structure_m in structures_m:

    for s in [structure_m, convert_to_surround_name(name=structure_m, margin='200um')]:
    
        stack_m_spec = dict(name='atlasV7',
                   vol_type='score',
                   structure=s,
                    resolution='10.0um'
                   )

    #     stack_f_spec = dict(name=stack,
    #                        vol_type='score',
    #                        detector_id=detector_id,
    #                        structure=convert_to_original_name(structure_m),
    #                         resolution='10.0um'
    #                        )

        # local_alignment_spec = dict(stack_m=stack_m_spec, 
        #                       stack_f=stack_f_spec,
        #                       warp_setting=registration_setting)

        # DataManager.save_alignment_results_v3(transform_parameters=convert_transform_forms(transform=tf_atlas_to_subj, out_form='dict'),
        #                score_traj=aligner.scores,
        #                parameter_traj=aligner.Ts,
        #               alignment_spec=local_alignment_spec)

        # tf_atlas_to_subj = DataManager.load_alignment_results_v3(local_alignment_spec, what='parameters', out_form=(4,4))

        atlas_structure_wrt_canonicalAtlasSpace_atlasResol = \
        DataManager.load_original_volume_v2(stack_spec=stack_m_spec, bbox_wrt='canonicalAtlasSpace', structure=s)

#         aligned_structure_wrt_wholebrain_inputResol = \
#         transform_volume_v4(volume=atlas_structure_wrt_canonicalAtlasSpace_atlasResol,
#                             transform=tf_atlas_to_subj,
#                             return_origin_instead_of_bbox=True)

#         DataManager.save_transformed_volume_v2(volume=aligned_structure_wrt_wholebrain_inputResol, 
#                                                alignment_spec=alignment_spec,
#                                               structure=s,
#                                                upload_s3=False)

        ###############################


        aligned_structure_wrt_wholebrain_inputResol = \
        transform_volume_v4(volume=atlas_structure_wrt_canonicalAtlasSpace_atlasResol,
                            transform=init_T,
                            return_origin_instead_of_bbox=True)

#         DataManager.save_transformed_volume_v2(volume=aligned_structure_wrt_wholebrain_inputResol, 
#                                                alignment_spec=simpleGlobal_alignment_spec,
#                                               structure=s,
#                                                upload_s3=False)

/home/yuncong/MouseBrainAtlas/src/utilities/registration_utilities.py:1571: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(volume_m_aligned_to_f.dtype, np.float):
Interpolating/filling sparse volume: 0.05 seconds.
Interpolating/filling sparse volume: 0.09 seconds.


In [19]:
atlas_structure_wrt_canonicalAtlasSpace_atlasResol[1]+compute_centroid_3d(atlas_structure_wrt_canonicalAtlasSpace_atlasResol[0])

array([198.04543786,  66.57848514,  -1.5       ])

In [17]:
init_T

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.03245321e+03],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 4.14338454e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 4.48000000e+02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [16]:
compute_centroid_3d(aligned_structure_wrt_wholebrain_inputResol[0]) + aligned_structure_wrt_wholebrain_inputResol[1]

array([1230.23848538,  480.45666516,  446.5       ])